<a href="https://colab.research.google.com/github/palmcorp/AOSP-ICSAce/blob/master/MinimumReproducableworking_v28_118.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os,pdb
print ("Set up problem & globals")
TRY2=True
YAML2="data17.yaml"  # yaml file for data
YAML1='data5.yaml'
DATA17='data17.yaml'# flag for data yam
DATA17P="dataP17.yaml"
DATA17Q='data17Q.yaml'
YOLO17='yolov8n-pose.yaml'  # yaml file for model
YOLO17P='yolov8n-poseVp.yaml'
YOLO17Q='yolov8n-poseVQ.yaml'
DATA_YAML = 'data17Q.yaml'
MODEL_YAML = 'yolovn-poseVR.yaml'
if TRY2:
  print ("changing to 4,3 with different class order \n MODEL T uwses YOlov8 nc=6 hack")
  DATA_YAML = "dataT.yaml"
  MODEL_YAML= "modelT.yaml"
NUM_CLASS =5
print ("NUMBER OF CLASSES: ",NUM_CLASS, " note : this may differ from hacked setting in YAML nc=6")
EPOCHS=2  # simple 2 epoch
EPOCHS = 20  # updatd for v 22
EPOCHS = 100 # final prodution yo be get best & last
EPOCHS = 200 # FINAL prodution for version 24
EPOCHS = 250  # catch up to alidate and remove RT drop and with TEST vids
PROJ_BASE=os.path.join ("/content/")
VERSION ="v28"
MISSION=" Run Phase II images & annotations \n PLUS CLASS ORDER CORRECTED TFY nvc=6 WORKES"
MISSION+="\n FINAL RUN WITH "+str(EPOCHS)   #100"
print ("\nVERSION ", VERSION, "\n MISSION: ", MISSION, "\n")
!nvidia-smi
!pip install --upgrade pip
!pip install ultralytics
!pip install --upgrade torch ultralytics
import os, pdb
from ultralytics import YOLO
!pip install --upgrade torch ultralytics hub-sdk

Set up problem & globals
changing to 4,3 with different class order 
 MODEL T uwses YOlov8 nc=6 hack
NUMBER OF CLASSES:  5  note : this may differ from hacked setting in YAML nc=6

VERSION  v28 
 MISSION:   Run Phase II images & annotations 
 PLUS CLASS ORDER CORRECTED TFY nvc=6 WORKES
 FINAL RUN WITH 250 

Tue Oct 29 17:16:45 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4             

In [2]:
print ("dont need google drive")
'''
 from google.colab import drive
 drive.mount('/content/drive')
 !nvidia-smi
 '''
def PRINT_FILE (FP):
  ret = True
  if not os.path.exists (FP):
    print (FP, " doesnt exist")
    return False
  #pdb.set_trace()
  NL=0
  print ("\n FILE: ", FP, "\n")
  with open (FP, 'r') as inp:
    print (inp.read())
    #line=inp.read()
    #print ("[",NL,"] ", line)
    #NL+=1
  print ("\n")
  return True


dont need google drive


In [3]:
print ("Model Steps : load/train/recognize ")
'''
def POSE_LOAD(data_yaml_path):
  model = YOLO(data_yaml_path, verbose = True)
  model = YOLO('yolov8n-pose.pt')  # load a pretrained model (recommended for training)
  model = YOLO('yolov8n-pose.yaml').load('yolov8n-pose.pt')  # build from YAML and transfer weights
  return model
'''
def POSE_LOAD(data_yaml_path):
  print ("REVERTIN TO ORIGINAL SCRIPT")
  model = YOLO('yolov8n-pose.yaml').load('yolov8m-pose.pt')  # build from YAML and transfer weights
  #model = YOLO('yolov8n-pose.yaml').load('YOLO11l-pose.pt')
  print ("Using this yaml: for data yaml: ", data_yaml_path)
  pdb.set_trace()
  #model = YOLO(data_yaml_path).load('yolov8m-pose.pt')  # build from YAML and transfer weights
  return model


def POSE_TRAIN(model, model_yaml_path):
  model.train(data=model_yaml_path, epochs=EPOCHS, imgsz=640)
  return model

def POSE_VAL(best_path):
  if best_path == None:
    model = YOLO('yolov8n-pose.pt')
    metrics =2
  else:
    model = YOLO(best_path)
    # Validate the model
    metrics = model.val(data=best_path)  # no arguments needed, dataset and settings remembered
    metrics.box.map    # map50-95
    metrics.box.map50  # map50
    metrics.box.map75  # map75
    metrics.box.maps
  return metrics


Model Steps : load/train/recognize 


In [4]:

import os
YAML_DIR = os.path.join ("/content/yaml_files/")
xx=input (" workng directory setup= cr")
if xx=="":
  #YAML_DIR = os.path.join ("/content/yaml_files/")
  os.makedirs (YAML_DIR, exist_ok=True)
  YAML_FILES = (DATA_YAML,MODEL_YAML)
  xx= input ('upload files to'+ str(YAML_DIR)+"  " + str(YAML_FILES)+ " then cr")
  FLST = os.listdir(YAML_DIR)
  #print ("FIles in yaml_files: ", FLST)
  for FL in FLST:
    if ".yaml" in FL:
      ret=PRINT_FILE(os.path.join (YAML_DIR,FL))

 workng directory setup= cr
upload files to/content/yaml_files/  ('dataT.yaml', 'modelT.yaml') then cr

 FILE:  /content/yaml_files/dataT.yaml 

#keypoint data sets

# Data
#path: D:POSE/data
path: /content/datasets/echo-pose
train: images/train   # subsets to D
val:   images/val # relative to path

# Keypoints
kpt_shape: [4,3]  # number of keypoints, number of dim
flip_indx: [0,1,2,3,4]  # no flipping

#backbone:  uskng standard backbone
backbone:
  # [from, repeats, module, args]
  - [-1, 1, Conv, [64, 3, 2]]  # 0-P1/2
  - [-1, 1, Conv, [128, 3, 2]]  # 1-P2/4
  - [-1, 3, C2f, [128, True]]
  - [-1, 1, Conv, [256, 3, 2]]  # 3-P3/8
  - [-1, 6, C2f, [256, True]]
  - [-1, 1, Conv, [512, 3, 2]]  # 5-P4/16
  - [-1, 6, C2f, [512, True]]
  - [-1, 1, Conv, [1024, 3, 2]]  # 7-P5/32
  - [-1, 3, C2f, [1024, True]]
  - [-1, 1, SPPF, [1024, 5]]  # 9
#usn standard head

head:
  - [-1, 1, nn.Upsample, [None, 2, "nearest"]]
  - [[-1, 8], 1, Concat, [1]] # cat backbone P5
  - [-1, 3, C2, [768, False]] 

In [5]:
xx=input("create data directories ? cr=no")
import os, pdb
BASE=os.path.join("/content")
def ONE(BASE,dir_path):
  DP=os.path.join (BASE,dir_path)
  os.makedirs(DP, exist_ok=True)
  return
def SETUP_SUBS(BASE,SUB_LIST):
  for S in SUB_LIST:
    sub_dir=os.path.join (BASE,S)
    os.makedirs(sub_dir, exist_ok=True)
    print ('created: ', sub_dir)
  return
if xx=="":
  print ("skipping dir creations")
else:
  T=False
  if T: pdb.set_trace()
  DIR_LIST =('yaml_files','datasets')
  SUB_LIST =('images', 'labels')
  for d in DIR_LIST:ONE(BASE, d)
  SUB_BASE =os.path.join ('/content','datasets','echo-pose')
  os.makedirs(SUB_BASE, exist_ok=True)
  #DIR_PATHS=((os.path.join (SUB_BASE,'train')),(os.path.join (SUB_BASE,'valid')))
  DIR_PATHS=("train", "val")
  SETUP_SUBS(SUB_BASE,SUB_LIST)
  for d in SUB_LIST:
    BASE = os.path.join ("/content/datasets/echo-pose/",d)
    for D in DIR_PATHS:
      SETUP_SUBS(BASE,[D])
'''xx=input("cr after uploading zip files to DATA_ZIP")
if xx=="":
  print ("up load complete, current contgnts follow")
  IMAGE_LIST =IL= os.path.join ("/content/datasets/echo-pose/images/train")
  print ("echo-pose|images|train =\n", os.listdir(IL))
  LABEL_LIST = LL=os.path.join ("/content/datasets/echo-pose/labels/train")
  print ("echo-pose|images|train =\n", os.listdir(LL))
'''
print ("upload SPLIT ZIP INTO DATA_ZIP")
ZIP_DIR=os.path.join ("/content/DATA_ZIP")
os.makedirs (ZIP_DIR, exist_ok=True)
xx=input ("cr when data files uploaded to unzip or N to skip")
if xx=="":
  PROJ=os.getcwd()
  #pdb.set_trace()
  print ("DATA_ZIP created, upload zips")
  path ="/content/datasets/echo-pose/images/train"
  os.chdir(path)
  !unzip /content/DATA_ZIP/TRAINimages.zip -x /content/datasets/echo-pose/images/train
  path = "/content/datasets/echo-pose/images/val"
  os.chdir(path)
  !unzip /content/DATA_ZIP/VALimages.zip   -x /content/datasets/echo-pose/images/val
  path="/content/datasets/echo-pose/labels/train"
  os.chdir(path)
  !unzip /content/DATA_ZIP/TRAINlabels.zip -x /content/datasets/echo-pose/labels/train
  path="/content/datasets/echo-pose/labels/val"
  os.chdir(path)
  !unzip /content/DATA_ZIP/VALlabels.zip   -x /content/datasets/echo-pose/labels/val
  print ("restoring root to ", PROJ)
  os.chdir(PROJ)
  xx=input ("count all contents cr = no")

  if xx!="":
    lstTrain = os.listdir("/content/datasets/echo-pose/images/train")
    nlT = len(lstTrain)
    lstVal   = os.listdir("/content/datasets/echo-pose/images/val")
    nlL = len(lstVal)
    LLT  = os.listdir ("/content/datasets/echo-pose/labels/train")
    nLT  = len(LLT)
    LLV  = os.listdir ("/content/datasets/echo-pose/labels/val")
    nLV  = len (LLV)
    print ("file count:\n # Train Images / Labels")
    print (nlT, nLT, "\n Val Images / labels: \n",nlL,nLV)


create data directories ? cr=noyes
created:  /content/datasets/echo-pose/images
created:  /content/datasets/echo-pose/labels
created:  /content/datasets/echo-pose/images/train
created:  /content/datasets/echo-pose/images/val
created:  /content/datasets/echo-pose/labels/train
created:  /content/datasets/echo-pose/labels/val
upload SPLIT ZIP INTO DATA_ZIP
cr when data files uploaded to unzip or N to skip
DATA_ZIP created, upload zips
Archive:  /content/DATA_ZIP/TRAINimages.zip
  inflating: AA094-000_M.jpg         
  inflating: AA094-001_M.jpg         
  inflating: AA094-002_M.jpg         
  inflating: AA094-004_M.jpg         
  inflating: AA094-005_M.jpg         
  inflating: AA094-006_M.jpg         
  inflating: AA094-007_M.jpg         
  inflating: AA094-008_M.jpg         
  inflating: AA094-009_M.jpg         
  inflating: AA094-010_M.jpg         
  inflating: AA094-011_M.jpg         
  inflating: AA094-014_M.jpg         
  inflating: AA094-017_M.jpg         
  inflating: AA094-018_M.j

In [6]:
xx=input ("set up Best ? cr = no")
if xx!="":
  import os
  WEIGHTSp= os.path.join (PROJ_BASE,"WEIGHTS")
  os.makedirs(WEIGHTSp, exist_ok=True)
  xx=input ("cr when best & last have been uploaded to WEIGHTS")
  print ("contents of WEIGHTS: ",os.listdir(WEIGHTSp))

set up Best ? cr = noyes
cr when best & last have been uploaded to WEIGHTS
contents of WEIGHTS:  ['best.pt', 'last.pt']


In [7]:
print ("short start with Paula")
#Data_Yaml_Path = os.path.join (PROJ_BASE,YAML_DIR, DATA17)
#Data_Yaml_Path = os.path.join (PROJ_BASE,YAML_DIR, DATA17P)
Data_Yaml_Path = os.path.join (PROJ_BASE,YAML_DIR, DATA_YAML)
if not os.path.isfile(Data_Yaml_Path):
  pdb.set_trace()
print ("using data yaml: ", Data_Yaml_Path)
NO_WANDB=False
#wandb disabled
if NO_WANDB:
  !pip uninstall wandb
  print ("UNINSTLL WANDB")
model=POSE_LOAD (Data_Yaml_Path)

pdb.set_trace()

#Model_Yaml_Path =os.path.join (PROJ_BASE,YAML_DIR,YOLO17)
#Model_Yaml_Path =os.path.join (PROJ_BASE, YAML_DIR,YOLO17P)
Model_Yaml_Path =os.path.join (PROJ_BASE, YAML_DIR,MODEL_YAML)
if not os.path.isfile(Model_Yaml_Path):
  pdb.set_trace()
print ("using YAML: ", Model_Yaml_Path)

POSE_TRAIN (model,Model_Yaml_Path)

short start with Paula
using data yaml:  /content/yaml_files/dataT.yaml
REVERTIN TO ORIGINAL SCRIPT


100%|██████████| 50.8M/50.8M [00:01<00:00, 35.9MB/s]

Transferred 133/397 items from pretrained weights




PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/lib/python3.10/bdb.py", line 336, in set_trace
    sys.settrace(self.trace_dispatch)



Using this yaml: for data yaml:  /content/yaml_files/dataT.yaml
> <ipython-input-3-e46bf81e4a49>(16)POSE_LOAD()
     14   pdb.set_trace()
     15   #model = YOLO(data_yaml_path).load('yolov8m-pose.pt')  # build from YAML and transfer weights
---> 16   return model
     17 
     18 

ipdb> c


...

                   all         92        184      0.793      0.772      0.816      0.489       0.87      0.853      0.932      0.787



250 epochs completed in 0.259 hours.
Optimizer stripped from runs/pose/train/weights/last.pt, 6.5MB
Optimizer stripped from runs/pose/train/weights/best.pt, 6.5MB

Validating runs/pose/train/weights/best.pt...
WARNING ⚠️ validating an untrained model YAML will result in 0 mAP.
Ultralytics 8.3.24 🚀 Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
YOLOv8n-pose summary (fused): 187 layers, 3,078,950 parameters, 0 gradients, 8.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Pose(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:01<00:00,  1.77it/s]


                   all         92        184      0.774      0.793      0.812      0.498      0.874      0.902      0.941      0.798
               class_0         92         92      0.786      0.772      0.797      0.523      0.892       0.88      0.948      0.799
               class_5         92         92      0.762      0.815      0.827      0.474      0.857      0.924      0.935      0.796
Speed: 0.3ms preprocess, 1.7ms inference, 0.0ms loss, 1.5ms postprocess per image
Results saved to runs/pose/train


YOLO(
  (model): PoseModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=

In [8]:
#nvidia-smi

In [9]:
FROM="Paula email 6-16-24"
import os, pdb
VESION= "0.3"

KEYSIZE=3  # FOR  2D keys, otherwise 4
def verify_labels(label_dir, expected_columns=35):
  print ("chcking labels for: ", label_dir)
  for label_file in os.listdir(label_dir):
      if label_file.endswith('.txt'):
        with open(os.path.join(label_dir, label_file), 'r') as f:
          for line in f:
            columns = line.strip().split()
            print ("class= ", columns[0])
            print ("B BOX= ", columns[1:5])
            HEADER = 5 # columns
            # remove header from len of line
            NCOL =int (len(columns[5:len(columns)]))
            print (NCOL , "columns after header")
            Nkeypoints =int (len(columns[5:len(columns)])/KEYSIZE)
            KeyColumns = int(NCOL/KEYSIZE)
            TotalColumns= Nkeypoints * KEYSIZE + HEADER
            #if Nkeypoints != (expected_columns-HEADER):
            if TotalColumns != expected_columns:
	                        print(f"Label file {label_file} has {len(columns)-HEADER} columns, expected {expected_columns} columns.")
	                        return False
  return True

pdb.set_trace()
print (" Label checker V", VERSION, " FROM: ", FROM, "\n")
label_dir=ld = '/content/datasets/echo-pose/labels/train'
if not os.path.exists (ld):
    print (ld, "not")
    pdb.set_trace()
if verify_labels(label_dir, 14):
	    print("All label files are correctly formatted.")
else:
            print("Some label files are incorrectly formatted.")
#pdb.set_trace()

--Return--
None
> <ipython-input-9-d70801afdbab>(28)<cell line: 28>()
     26   return True
     27 
---> 28 pdb.set_trace()
     29 print (" Label checker V", VERSION, " FROM: ", FROM, "\n")
     30 label_dir=ld = '/content/datasets/echo-pose/labels/train'

ipdb> c
 Label checker V v28  FROM:  Paula email 6-16-24 

chcking labels for:  /content/datasets/echo-pose/labels/train
class=  0
B BOX=  ['0.683489', '0.571071', '0.0771304', '0.0563152']
12 columns after header
Label file E8F3E-17_M.txt has 12 columns, expected 14 columns.
Some label files are incorrectly formatted.


In [10]:
print ("skip this")
'''print ("load model for test / evaluation")
def POSE_TEST(TEST_DIR,  best_path = os.path.join ("/content/WEIGHTS/best.pt")):
  xx=input ("cr when best loaded into WEIGHTS")

#def POSE_VAL(best_path):
  if best_path == None:
    model = YOLO('yolov8n-pose.pt')
    metrics =2
    pdb.set_trace()
  else:
    model = YOLO(best_path)
    # Validate the model
   #metrics = model.val(data=best_path)  # no arguments needed, dataset and settings remembered
    megrics = model.val(data=TEST_DIR)
    metrics.box.map    # map50-95
    metrics.box.map50  # map50
    metrics.box.map75  # map75
    metrics.box.maps
  return metrics
  '''

skip this


'print ("load model for test / evaluation")\ndef POSE_TEST(TEST_DIR,  best_path = os.path.join ("/content/WEIGHTS/best.pt")):\n  xx=input ("cr when best loaded into WEIGHTS")\n\n#def POSE_VAL(best_path):\n  if best_path == None:\n    model = YOLO(\'yolov8n-pose.pt\')\n    metrics =2\n    pdb.set_trace()\n  else:\n    model = YOLO(best_path)\n    # Validate the model\n   #metrics = model.val(data=best_path)  # no arguments needed, dataset and settings remembered\n    megrics = model.val(data=TEST_DIR)\n    metrics.box.map    # map50-95\n    metrics.box.map50  # map50\n    metrics.box.map75  # map75\n    metrics.box.maps\n  return metrics\n  '

In [11]:
def PICK_ONE (LIST):
      picks=""
      npicks = len(LIST)
      for i in range (npicks):
          print ("[",i,"]=", LIST[i])
      xx=input ("PIck from list by number: ")
      try:
          pick=int(xx)
      except Exception as Ex:
          print ("exception is Pick one: ", Ex)
          xx= input ("try a number")
          pick= int(xx)
      if pick>npicks :
          print ("not in list: ", pick)
          pdb.set_trace()
      pick_value = LIST[pick]
      pick_number= pick
      return pick_number, pick_value
  # filter out extensions
def PICK_ONEX(LIST, FORMAT=""):
    if FORMAT=="":
        return (PICK_ONE(LIST)) # no fileter necesary
    xLIST=[]
    for FN in LIST:
        if FORMAT in FN:
            xLIST.append (FN)
    return PICK_ONE (xLIST)


In [32]:
print ("evalute some images")
import os, pdb
# zip them
pdb.set_trace()
import pickle
def SaveR(DATA, NAME):
  N=NAME
  # v2 use pickle
  PN=NAME +".pkl"
  try:
    fp=os.path.join ("/content/ZIPS/RESULTS", PN)
    with open (fp, "wb")as handle:
      pickle.dump(DATA, handle,protocol=pickle.HIGHEST_PROTOCOL)
    print (NAME, "saved as pickle at ", fp)
    return True
  except Exception as EX:
      print ("Saver Ex:", EX)
      return False
  '''
  file = os.open(fp,"rb")
  file.write (str(DATA))
  file.close()
  return
  '''
pdb.set_trace()

import os, pickle
BASE = os.path.join ("/content/datasets/echo-pose/images")
SUB2  ="val"
SUB1="train"
RESULTS=R=os.path.join("/content/ZIPS", "RESULTS")
os.makedirs(R, exist_ok=True)
xx=input ('skip evaluation of trained images cr=yes')
if xx=="":
  print ("skipping")
  SKIP=True
else:
  SKIP=False
  print ("using 2nd generation images AA094 and E8F3E  take first and last")
  pSUB2=os.path.join (BASE, SUB2)
  ListSUB2= os.listdir(pSUB2)
  pSUB1=os.path.join (BASE, SUB1)
  ListSUB1=os.listdir(pSUB1)
  FIRST=ListSUB1[0]
  LAST =ListSUB2[len(ListSUB2)-1]
  print ("using ", FIRST, " and ", LAST)
  img1P=os.path.join (BASE,SUB1,FIRST)
  img2P=os.path.join (BASE,SUB2,LAST)
  #img1P=os.path.join (BASE,SUB1,"frame_000000.PNG")
  #img2P=os.path.join (BASE,SUB1,"000000000036.jpg")

  # Predict with the model
  results1 = model(img2P, save=True, save_txt=True)  # predict on an
  pdb.set_trace()
  results2 = model(img1P, save=True)
  pdb.set_trace()
  ret = SaveR(results1,"results1")
  ret = SaveR(results2,'results2')
  VImages= os.listdir (os.path.join (BASE,"val"))
  nImg=len(VImages)
xx=input ("eval all val images =cr")
if xx=="":
  BASEV=os.path.join (BASE,SUB2)
  print ("from BASE; ", BASEV)
  VImages= os.listdir (os.path.join (BASE,"val"))# use original
  nImg=len(VImages)
  N=10
  TEST=False
  R='results_'
  VF=['PNG', 'jpg']
  for IMG in VImages:
    #if not VF in IMG : continue
    if not any(x in IMG for x in VF): continue
    FP= os.path.join (BASEV,IMG)
    resultN=R+str(N)
    resultND=model (FP, save=True, save_txt=True)
    if TEST: pdb.set_trace()
    ret=SaveR(resultND,resultN)
    N+=1
  print ("saved ", N-10, " results")
xx=input ("evaluate all train imaegs = cr")
if xx=="":
    TImages= os.listdir (os.path.join (BASE,"train"))
    nImg=len(TImages)
    BASET=os.path.join (BASE, SUB1)
    print ("saving all from ", BASET)
    N=100
    R='results_'
    VF=['PNG', 'jpg']
    for IMG in TImages:
      if not any(x in IMG for x in VF): continue
      FP= os.path.join (BASET,IMG)
      resultN=R+str(N)
      resultND=model (FP, save=True, save_txt=True)
      ret=SaveR(resultND,resultN)
      N+=1
    print ("saved ", N-100, " results")
xx=input("cr to run TEST videos, cr to start")
if xx=="":
  TEST_DIR = os.path.join ("/content/TEST")
  os.makedirs (TEST_DIR, exist_ok=True)
  LST_TEST=LST = os.listdir(TEST_DIR)
  THIS_SUB = "NORMAL"
  SUB_TEST= os.path.join (TEST_DIR,THIS_SUB)
  print ("testing ", SUB_TEST)
  os.makedirs(SUB_TEST, exist_ok=True)
  xx=input ("cr when ready to test from: "+str(SUB_TEST))
  LST_TEST = os.listdir(SUB_TEST)
  print (len(LST_TEST), " videos found, from subdir")
  #nx,pick = PICK_ONE(LST)
  #if nx<0: pdb.set_trace
  TEST_DIR_FP =TFP= SUB_TEST#  os.path.join (TEST_DIR, pick)
  #SUBX=pick
  #xx=input ("cr when videos are uploaded to" +str(TFP)# /content/TEST")
  LST_TEST =LLST= os.listdir(TFP)
  #print ("full path to test images ")
  print ("sub dir conrtents: ", str(LLST))
  pdb.set_trace()
  xx=input ( "load model? cr=SKIP")
  if xx=="":
    print ("skipping POSE_LOAD")
  else:
    Data_Yaml_Path = os.path.join (PROJ_BASE,YAML_DIR, DATA_YAML) # see aboe
    data_yaml_path = Data_Yaml_Path = os.path.join (PROJ_BASE,YAML_DIR, DATA_YAML)
    model =POSE_LOAD(data_yaml_path)
    print ("model loaded : ", type(model))
  N=200
  R='results_'
  TImages = LST_TEST
  VF=['PNG', 'jpg', '.mp4','.avi']
  #SUBX="SEVERE"
  #SUBX=pick
  BASEQ=TFP #os.path.joion (TEST_DIR,SUBX)
  if not os.path.exists (BASEQ):
    print (BASEQ, " does not exist")
    pdb.set_trace()
  else:
    print ("loading images from : ", BASEQ)

  for IMG in TImages:
      if not any(x in IMG for x in VF): continue
      FP= os.path.join (BASEQ,IMG)
      resultN=R+str(N)
      resultND=model (FP, save=True, save_txt=True)
      ret=SaveR(resultND,resultN)
      N+=1
  print ("END OF TEST VIDS")
  pdb.set_trace()

Streaming output truncated to the last 5000 lines.
video 1/1 (frame 41/131) /content/TEST/NORMAL/0X44A3EA77AFCBC602_N.mp4: 640x640 (no detections), 8.0ms
video 1/1 (frame 42/131) /content/TEST/NORMAL/0X44A3EA77AFCBC602_N.mp4: 640x640 (no detections), 8.1ms
video 1/1 (frame 43/131) /content/TEST/NORMAL/0X44A3EA77AFCBC602_N.mp4: 640x640 (no detections), 8.2ms
video 1/1 (frame 44/131) /content/TEST/NORMAL/0X44A3EA77AFCBC602_N.mp4: 640x640 (no detections), 8.3ms
video 1/1 (frame 45/131) /content/TEST/NORMAL/0X44A3EA77AFCBC602_N.mp4: 640x640 (no detections), 8.3ms
video 1/1 (frame 46/131) /content/TEST/NORMAL/0X44A3EA77AFCBC602_N.mp4: 640x640 (no detections), 8.1ms
video 1/1 (frame 47/131) /content/TEST/NORMAL/0X44A3EA77AFCBC602_N.mp4: 640x640 (no detections), 8.1ms
video 1/1 (frame 48/131) /content/TEST/NORMAL/0X44A3EA77AFCBC602_N.mp4: 640x640 (no detections), 8.5ms
video 1/1 (frame 49/131) /content/TEST/NORMAL/0X44A3EA77AFCBC602_N.mp4: 640x640 (no detections), 8.0ms
video 1/1 (frame 50/13

In [31]:
print (" emptry data sets BE CAREFUL!!")
#D1=os.path.join ("//content/datasets/echo-pose/images/train)
!rm -r /content/datasets/echo-pose/images/train
!rm -r /content/datasets/echo-pose/images/val
!rm -r /content/TEST/NORMAL
!rm -r /content/runs/pose/


 emptry data sets
rm: cannot remove '/content/datasets/echo-pose/images/train': No such file or directory
rm: cannot remove '/content/datasets/echo-pose/images/val': No such file or directory
rm: cannot remove '/content/TEST/NORMAL': No such file or directory


In [13]:
print  ("settin up utf-8")
import locale
locale.getpreferredencoding = lambda: "UTF-8"
print ("see:https://stackoverflow.com/questions/56081324/why-are-google-colab-shell-commands-not-working ")

settin up utf-8
see:https://stackoverflow.com/questions/56081324/why-are-google-colab-shell-commands-not-working 


In [33]:
xx=input ("capture all data=cr")
ZIPS=os.path.join ("/content/ZIPS")
os.makedirs(ZIPS, exist_ok=True)
if xx=="":
  ZIPS=os.path.join ("/content/ZIPS")
  os.makedirs(ZIPS, exist_ok=True)
  !zip -r /content/ZIPS/AllData.zip /content/datasets
xx= input ("all runs? cr ")
if xx =="":
  !zip -r /content/ZIPS/AllRuns.zip /content/runs
xx=input ("all results = cr")
if xx=="":
  !zip -r /content/ZIPS/AllResults.zip /content/ZIPS/RESULTS
print ("CAPTURES ALL DONE: LOOK IN ZIPS")

capture all data=crno
all runs? cr 
  adding: content/runs/ (stored 0%)
  adding: content/runs/pose/ (stored 0%)
  adding: content/runs/pose/train17/ (stored 0%)
  adding: content/runs/pose/train17/0X44ADA67E755F8482_N.avi (deflated 26%)
  adding: content/runs/pose/train17/labels/ (stored 0%)
  adding: content/runs/pose/train17/labels/0X44ADA67E755F8482_N_137.txt (deflated 43%)
  adding: content/runs/pose/train17/labels/0X44ADA67E755F8482_N_138.txt (deflated 44%)
  adding: content/runs/pose/train17/labels/0X44ADA67E755F8482_N_143.txt (deflated 43%)
  adding: content/runs/pose/train17/labels/0X44ADA67E755F8482_N_130.txt (deflated 41%)
  adding: content/runs/pose/train17/labels/0X44ADA67E755F8482_N_144.txt (deflated 42%)
  adding: content/runs/pose/train17/labels/0X44ADA67E755F8482_N_47.txt (deflated 43%)
  adding: content/runs/pose/train17/labels/0X44ADA67E755F8482_N_99.txt (deflated 44%)
  adding: content/runs/pose/train17/labels/0X44ADA67E755F8482_N_135.txt (deflated 43%)
  adding: co

In [15]:
xx=input ('run model? Y = yes')
if xx=="Y":
  print ("Data YAML = ", DATA_YAML)
  #data_yaml_path = os.path.join ("/content/yaml_files/data5.yaml")
  data_yaml_path = os.path.join ("/content/yaml_files/",DATA_YAML)
  print ("adjusting yaml to 2nd Generations",data_yaml_path)
  WEIGHTS_DIR = "/content/WEIGHTS/"
  os.makedirs(WEIGHTS_DIR, exist_ok=True)
  best_path= os.path.join (WEIGHTS_DIR, "best.pt")
  print ("created ", best_path, " upload best.pt and last.pt then cr")
  xx=input ("cr when best uploaded")
  if xx=="": print ("running POSE Model")

  model=POSE_LOAD(data_yaml_path)
  model=POSE_TRAIN(model, data_yaml_path)
  #best_path = os.path.join ()
  POSE_VAL(best_path)
else:
   print ("skipped")

run model? Y = yesno
skipped


In [16]:
print ("not valid dnow")
#!unzip -o /content/UPZIP/images.zip  -d /content/data/train/images/
#!unzip -o /content/UPZIP/labels.zip   -d /content/data/train/label/

not valid dnow
